In [0]:
%run ../utils/common

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_pog_store_sku (
  store_concept STRING,
  store_id STRING,
  product_id STRING,
  assortment STRING,
  assortment_class_desc STRING,
  mdq STRING,
  approval_status STRING,
  winare_mdq STRING,
  external_report_mdq STRING,
  mdq_source STRING
)
USING DELTA
TBLPROPERTIES (
'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_pog_store_sku
AS
WITH portal_concept_asm_sku AS (
    SELECT DISTINCT
        a1.store_concept,
        a.store_id, 
        a.product_id,
        a.assortment,
        a.assortment_class_desc,
        a1.mdq,
        a1.approval_status,
        IF(a1.approval_status = "YES", a1.mdq, NULL) AS winare_mdq,
        IF(a1.applied_external_report = "YES", a1.mdq, NULL) AS external_report_mdq,
        'POG MDQ' mdq_source
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_pog a1
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd a
        ON a1.product_id = a.product_id 
        AND a.assortment LIKE 'UB0%'
        AND NOT(UPPER(a.assortment) RLIKE 'HERO')
        AND a1.core_range = a.assortment_class_desc
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c 
        ON a.store_id = c.store_id
    WHERE a.status = 'ACTIVE'
        AND a1.store_concept = 'URBAN'
        AND UPPER(IFNULL(c.concept, 'NULL')) <> 'RURAL'
        AND c.business_unit = '2000'
        AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN DATE(a.start_date) AND DATE(a.end_date)
    QUALIFY ROW_NUMBER() OVER(PARTITION BY a.store_id, a.product_id ORDER BY a.assortment_class_desc) = 1
    UNION ALL
    SELECT DISTINCT
        a1.store_concept,
        a.store_id, 
        a.product_id, 
        a.assortment,
        a.assortment_class_desc,
        a1.mdq,
        a1.approval_status,
        IF(a1.approval_status = "YES", a1.mdq, NULL) AS winare_mdq,
        IF(a1.applied_external_report = "YES", a1.mdq, NULL) AS external_report_mdq,
        'POG MDQ' mdq_source
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_pog a1
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd a
        ON a1.product_id = a.product_id 
        AND  (UPPER(a.assortment) LIKE 'R%CORE%' OR UPPER(a.assortment) LIKE 'R%HERO%')
        AND a1.core_range = a.assortment_class_desc
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c
        ON a.store_id = c.store_id
    WHERE a.status = 'ACTIVE'
        AND a1.store_concept = 'RURAL'
        AND UPPER(c.concept) = 'RURAL'
        AND c.business_unit = '2000'
        AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN DATE(a.start_date) AND DATE(a.end_date)
    QUALIFY ROW_NUMBER() OVER(PARTITION BY a.store_id, a.product_id ORDER BY a.assortment_class_desc) = 1
),
portal_store_sku AS (
    SELECT
        IF(UPPER(IFNULL(b.concept, 'NULL')) = 'RURAL', 'RURAL', 'URBAN') store_concept,
        a.store_id, 
        a.product_id,
        c.assortment,
        c.assortment_class_desc,
        a.mdq,
        a.approval_status,
        IF(a.approval_status = "YES", a.mdq, NULL) AS winare_mdq,
        NULL external_report_mdq,
        'POG MDQ ADHOC' mdq_source
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_store_sku_adhoc a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b
        USING(store_id)
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd c
        ON a.store_id = c.store_id 
            AND a.product_id = c.product_id
    WHERE DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN a.start_date AND a.end_date
        AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN DATE(c.start_date) AND DATE(c.end_date)
    QUALIFY ROW_NUMBER() OVER(PARTITION BY a.store_id, a.product_id ORDER BY c.assortment_class_desc NULLS LAST) = 1
),
portal AS (
    SELECT 
        IFNULL(b.store_concept, a.store_concept) store_concept,
        IFNULL(b.store_id, a.store_id) store_id,
        IFNULL(b.product_id, a.product_id) product_id,
        IFNULL(b.assortment, a.assortment) assortment,
        IFNULL(b.assortment_class_desc, a.assortment_class_desc) assortment_class_desc,
        IFNULL(b.mdq, a.mdq) mdq,
        IFNULL(b.approval_status, a.approval_status) approval_status,
        IFNULL(b.winare_mdq, a.winare_mdq) winare_mdq,
        IFNULL(b.external_report_mdq, a.external_report_mdq) external_report_mdq,
        IFNULL(b.mdq_source, a.mdq_source) mdq_source
    FROM portal_concept_asm_sku a
    FULL JOIN portal_store_sku b
        USING(store_id, product_id)
),
approve_core3 AS (
    SELECT 
        a.store_concept,
        a.store_id, 
        a.product_id, 
        a.assortment,
        a.assortment_class_desc,
        a.mdq,
        CASE
            WHEN a.assortment_class_desc = 'CORE3' AND a.approval_status = "YES" AND b.store_id IS NULL THEN "YES"
            WHEN a.assortment_class_desc = 'CORE3' AND a.approval_status = "YES" AND b.store_id IS NOT NULL THEN "NO"
            ELSE a.approval_status
        END approval_status,
        CASE
            WHEN a.assortment_class_desc = 'CORE3' AND a.approval_status = "YES" AND b.store_id IS NULL THEN a.mdq
            WHEN a.assortment_class_desc = 'CORE3' AND a.approval_status = "YES" AND b.store_id IS NOT NULL THEN NULL
            ELSE a.winare_mdq
        END winare_mdq,
        a.external_report_mdq,
        a.mdq_source
    FROM portal a
    LEFT JOIN (
        SELECT store_id, product_id
        FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_dynamic_assorment
        WHERE is_keeping = 0
    ) B
        ON a.store_id = b.store_id AND a.product_id = b.product_id
),
main AS (
    SELECT *
    FROM approve_core3
)
SELECT *
FROM main
""")

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_pog_store_sku_his (
  calday DATE,
  store_concept STRING,
  store_id STRING,
  product_id STRING,
  assortment STRING,
  assortment_class_desc STRING,
  mdq DOUBLE,
  approval_status STRING,
  winare_mdq DOUBLE,
  external_report_mdq DOUBLE,
  mdq_source STRING
)
USING DELTA
TBLPROPERTIES (
'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
);          
""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_pog_store_sku_his WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS);          
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_pog_store_sku_his
SELECT
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) calday,
    store_concept,
    store_id,
    product_id,
    assortment,
    assortment_class_desc,
    mdq,
    approval_status,
    winare_mdq,
    external_report_mdq,
    mdq_source
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_pog_store_sku;
""")